In [1]:
import import_ipynb
import preprocessor

importing Jupyter notebook from preprocessor.ipynb


In [2]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

In [3]:
# Train set, Test set
df = preprocessor.preprocessed
train_set, test_set = df.iloc[:-200, :], df.iloc[-200:, :]
train_set.shape, test_set.shape
list(enumerate(df.columns))

[(0, '사망자수'),
 (1, '중상자수'),
 (2, '경상자수'),
 (3, '부상신고자수'),
 (4, '주야_야간'),
 (5, '주야_주간'),
 (6, '요일_금'),
 (7, '요일_목'),
 (8, '요일_수'),
 (9, '요일_화'),
 (10, '요일_월'),
 (11, '요일_일'),
 (12, '요일_토'),
 (13, '발생지시도_경기'),
 (14, '발생지시도_전남'),
 (15, '발생지시도_충남'),
 (16, '발생지시도_대구'),
 (17, '발생지시도_서울'),
 (18, '발생지시도_광주'),
 (19, '발생지시도_부산'),
 (20, '발생지시도_충북'),
 (21, '발생지시도_강원'),
 (22, '발생지시도_경남'),
 (23, '발생지시도_인천'),
 (24, '발생지시도_세종'),
 (25, '발생지시도_경북'),
 (26, '발생지시도_전북'),
 (27, '발생지시도_대전'),
 (28, '발생지시도_울산'),
 (29, '발생지시도_제주'),
 (30, '발생지시군구_성남시'),
 (31, '발생지시군구_곡성군'),
 (32, '발생지시군구_서산시'),
 (33, '발생지시군구_서구'),
 (34, '발생지시군구_영등포구'),
 (35, '발생지시군구_달성군'),
 (36, '발생지시군구_용인시'),
 (37, '발생지시군구_광양시'),
 (38, '발생지시군구_광산구'),
 (39, '발생지시군구_중구'),
 (40, '발생지시군구_음성군'),
 (41, '발생지시군구_화성시'),
 (42, '발생지시군구_원주시'),
 (43, '발생지시군구_고양시'),
 (44, '발생지시군구_논산시'),
 (45, '발생지시군구_동구'),
 (46, '발생지시군구_고성군'),
 (47, '발생지시군구_안산시'),
 (48, '발생지시군구_목포시'),
 (49, '발생지시군구_강서구'),
 (50, '발생지시군구_시흥시'),
 (51, '발생지시군구_이천시'),
 (52, '발생지시군구_사상구'),
 (53, '

In [11]:
# 마지막 행 예측 문제

def drop_column_with_indices(df, predicting_column_indices):
    return df.drop(df.columns[predicting_column_indices], axis=1)

def divide_sets(predicting_column_indices):
    x_data = drop_column_with_indices(train_set, predicting_column_indices).values
    y_data = train_set.iloc[:, predicting_column_indices].values
    x_test = drop_column_with_indices(test_set, predicting_column_indices).values
    y_test = test_set.iloc[:, predicting_column_indices].values
    return x_data, y_data, x_test, y_test

In [12]:
def get_batch(step, batch_size, x_data, y_data):
    training_data_size = x_data.shape[0]
    start_idx = step * batch_size
    end_idx = (step + 1) * batch_size
    if end_idx > training_data_size:
        end_idx = training_data_size
    return (
        x_data[start_idx:end_idx, :],
        y_data[start_idx:end_idx, :]
    )

In [28]:
def build_network(layer_sizes, learning_rate):
    X = tf.placeholder(tf.float32, shape=[None, layer_sizes[0]])
    Y = tf.placeholder(tf.float32, shape=[None, layer_sizes[-1]])

    W1 = tf.Variable(tf.random_normal([layer_sizes[0], layer_sizes[1]]))
    b1 = tf.Variable(tf.random_normal([layer_sizes[1]]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    W2 = tf.Variable(tf.random_normal([layer_sizes[1], layer_sizes[2]]))
    b2 = tf.Variable(tf.random_normal([layer_sizes[2]]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

    W3 = tf.Variable(tf.random_normal([layer_sizes[2], layer_sizes[-1]]))
    b3 = tf.Variable(tf.random_normal([layer_sizes[-1]]))
    hypothesis = tf.matmul(L2, W3) + b3

    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=hypothesis, labels=Y
        ))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost)
    
    return train, cost, hypothesis, X, Y

In [29]:
def run(training_data_size, batch_size, training_epochs, cost, train, X, Y):
    total_batch = int(training_data_size / batch_size)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0
        for step in range(total_batch):
            batch_x, batch_y = get_batch(step, batch_size, x_data, y_data)
            cost_val, _ = sess.run(
                [cost, train],
                feed_dict={X: batch_x, Y: batch_y}
            )
            avg_cost += cost_val / total_batch
        print('Epoch:', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
    return sess

In [31]:
def calc_accuracy(hypothesis, X, Y, x_test, y_test):
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={X: x_test, Y: y_test})
    print('Accuracy:', result)

In [61]:
predicting_column_indices = [4, 5]
learning_rate = 2e-2
training_epochs = 10
batch_size = 100

x_data, y_data, x_test, y_test = divide_sets(predicting_column_indices)
training_data_size = x_data.shape[0]
layer_sizes = [x_data.shape[1], 300, 200, y_data.shape[1]]
train, cost, hypothesis, X, Y = build_network(layer_sizes, learning_rate)
sess = run(training_data_size, batch_size, training_epochs, cost, train, X, Y)
calc_accuracy(hypothesis, X, Y, x_test, y_test)

Epoch: 0001 cost =  43.184712064
Epoch: 0002 cost =  6.253242654
Epoch: 0003 cost =  2.838499060
Epoch: 0004 cost =  1.922712848
Epoch: 0005 cost =  1.510172726
Epoch: 0006 cost =  0.974818192
Epoch: 0007 cost =  0.782955785
Epoch: 0008 cost =  0.762762557
Epoch: 0009 cost =  0.603979669
Epoch: 0010 cost =  0.543501629
Accuracy: 0.555
